In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [3]:
import sys
import os

sys.path.append('../tools')
import h5py
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
from torch import nn
from tqdm import tqdm
import sklearn
from sklearn.model_selection import train_test_split
import torchvision.transforms as T
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks
from models.kaggle_dog_dataloader import *
from models.SwaV import SwaV
from models.SupervisedDownstream import SupervisedDownstream
from active_learning_data import Data
from sklearn.metrics import roc_curve, auc

In [4]:
data_dir = "../../../user_data/competition_data/clips"
log_folder_root = '../../../user_data/logs/'
ckpt_folder_root = '../../../user_data/checkpoints/'

targets = [
    'Dog_1',
    'Dog_2',
    'Dog_3',
    'Dog_4',
]

# targets = [
#     'Patient_1',
#     'Patient_2',
#     'Patient_3',
#     'Patient_4',
#     'Patient_5',
#     'Patient_6',
#     'Patient_7',
#     'Patient_8'
# ]

In [5]:
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
determine_generator = torch.Generator()
determine_generator.manual_seed(random_seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    # True ensures the algorithm selected by CUFA is deterministic
    torch.backends.cudnn.deterministic = True
    # torch.set_deterministic(True)
    # False ensures CUDA select the same algorithm each time the application is run
    torch.backends.cudnn.benchmark = False

In [6]:
def translate_prediction(prediction, y_classes):
    if len(prediction) == 3:
        # S is 1.0 when ictal <=15 or >15
        # S is 0.0 when interictal is highest
        ictalLTE15, ictalGT15, interictal = prediction
        S = ictalLTE15 + ictalGT15
        E = ictalLTE15
        return S, E
    elif len(prediction) == 2:
        # 1.0 doesn't exist for Patient_4, i.e. there is no late seizure data
        if not np.any(y_classes == 1.0):
            ictalLTE15, interictal = prediction
            S = ictalLTE15
            E = ictalLTE15
            # y[i] = 0 # ictal <= 15
            # y[i] = 1 # ictal > 15
            # y[i] = 2 # interictal
            return S, E
        else:
            raise NotImplementedError()
    else:
        raise NotImplementedError()


# use the classifier and make predictions on the test data
def make_predictions(target, X_test, y_classes, classifier_data):
    classifier = classifier_data.classifier
    predictions_proba = classifier.predict_proba(X_test)

    lines = []
    for i in range(len(predictions_proba)):
        p = predictions_proba[i]
        S, E = translate_prediction(p, y_classes)
        lines.append('%s_test_segment_%d.mat,%.15f,%.15f' % (target, i+1, S, E))

    return {
        'data': '\n'.join(lines)
    }


# the scoring mechanism used by the competition leaderboard
def score_classifier_auc(classifier, X_cv, y_cv, y_classes):
    predictions = classifier.predict_proba(X_cv)
    S_predictions = []
    E_predictions = []
    S_y_cv = [1.0 if (x == 0.0 or x == 1.0) else 0.0 for x in y_cv]
    E_y_cv = [1.0 if x == 0.0 else 0.0 for x in y_cv]

    for i in range(len(predictions)):
        p = predictions[i]
        S, E = translate_prediction(p, y_classes)
        S_predictions.append(S)
        E_predictions.append(E)

    fpr, tpr, thresholds = roc_curve(S_y_cv, S_predictions)
    S_roc_auc = auc(fpr, tpr)
    fpr, tpr, thresholds = roc_curve(E_y_cv, E_predictions)
    E_roc_auc = auc(fpr, tpr)

    return S_roc_auc, E_roc_auc

In [7]:

test_data_list = [kaggle_data_utility.parse_input_data(data_dir, targets[i], 'test', None) for i in range(len(targets))]

Loading data
(1s)
X (3181, 16, 400)
Loading data
(1s)
X (2997, 16, 400)
Loading data
(1s)
X (4450, 16, 400)
Loading data
(1s)
X (3013, 16, 400)


In [8]:
test_data_X = np.empty((0, 16, 400))
for i in tqdm(range(len(test_data_list))):
    test_data = test_data_list[i]['X']

    test_data_X = np.vstack((test_data_X, test_data))

100%|██████████| 4/4 [00:00<00:00, 13.51it/s]


In [9]:
test_data = ActiveDataHandler(test_data_X,np.zeros(len(test_data_X))[:,np.newaxis])

In [10]:
# swav = SwaV().load_from_checkpoint(ckpt_folder_root + 'kaggle_dog_swav_18/kaggle_dog_swav-epoch=116-swav_loss=2.23095.ckpt')
swav = SwaV().load_from_checkpoint(
    ckpt_folder_root + 'kaggle_dog_swav_34/kaggle_dog_swav-epoch=116-swav_loss=2.73583.ckpt')
# swav = SwaV().load_from_checkpoint(ckpt_folder_root + 'kaggle_dog_swav_50/kaggle_dog_swav-epoch=118-swav_loss=3.64062.ckpt')
model = SupervisedDownstream(swav.backbone)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

checkpoint_callback = pl_callbacks.ModelCheckpoint(monitor='val_loss',
                                                   filename='kaggle_dog_linear_eval-{epoch:02d}-{val_loss:.5f}',
                                                   save_last=True, save_top_k=-1,
                                                   dirpath=ckpt_folder_root + 'kaggle_dog_swav_34_linear_eval_3c')
csv_logger = pl_loggers.CSVLogger(log_folder_root, name="kaggle_dog_swav_34_linear_eval_3c")

trainer = pl.Trainer(logger=csv_logger, max_epochs=80, callbacks=[checkpoint_callback], accelerator='gpu', devices=1)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
strategies = os.listdir(ckpt_folder_root+'kaggle_dog_active')
saved_ckpts = os.listdir(ckpt_folder_root+'kaggle_dog_active/'+ strategies[2])

In [12]:
def collate_fn(batch):
    info = list(zip(*batch))
    data = info[0]
    label = info[1]
    return torch.stack(data), torch.stack(label)

val_dataloader = torch.utils.data.DataLoader(
    test_data,
    batch_size=256,
    collate_fn=collate_fn,
    shuffle=False,
    drop_last=True,
    # num_workers= 4
)
# predictions = trainer.predict(model, val_dataloader,ckpt_path=ckpt_folder_root + 'kaggle_dog_active/active_checkpoints_EntropySampling/EntropySampling_round_12-epoch=99-train_loss=0.25522.ckpt')

In [13]:
labels = pd.read_csv('SzDetectionAnswerKey.csv')
result = pd.DataFrame(columns = ['Name', 'Score'])

In [14]:
import re
for stg in strategies:
    filenames = os.listdir(ckpt_folder_root+'kaggle_dog_active/'+ stg)
    pattern = r'round_(\d+)-epoch=(\d+)-train_loss'

    filtered_filenames = [
        filename for filename in filenames
        if int(re.search(pattern, filename).group(1)) >= 8 and int(re.search(pattern, filename).group(2)) >= 40
    ]
    
    # Print the filtered filenames
    for ckpt in filtered_filenames:
        try:
            predictions = trainer.predict(model, val_dataloader,ckpt_path=ckpt_folder_root + 'kaggle_dog_active/' + stg + '/' + ckpt)
            output_list = []
            emb_list = []
            m = nn.Softmax(dim=1)
            for pred, y, emb in predictions:
                output_list.append(pred)
                emb_list.append(emb)
            pred_raw = torch.vstack(output_list)
            emb = torch.vstack(emb_list)
            out = torch.argmax(pred_raw, dim=1)
            output = torch.vstack(output_list)
            S = output[:,0] + output[:,1]
            E = output[:,0]
            
            fpr, tpr, thresholds = roc_curve(labels.iloc[:13641][' seizure'], S)
            S_roc_auc = auc(fpr, tpr)
            fpr, tpr, thresholds = roc_curve(labels.iloc[:13641][' early '], E)
            E_roc_auc = auc(fpr, tpr)
            final_score = (S_roc_auc+E_roc_auc)/2
            result.loc[len(result.index)] = [ckpt, final_score]
            result.to_csv('result.csv')
        except:
            print('passed',ckpt)
            pass

Restoring states from the checkpoint path at ../../../user_data/checkpoints/kaggle_dog_active/active_checkpoints_BadgeSampling/BadgeSampling_round_10-epoch=49-train_loss=0.19825.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ../../../user_data/checkpoints/kaggle_dog_active/active_checkpoints_BadgeSampling/BadgeSampling_round_10-epoch=49-train_loss=0.19825.ckpt
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

Restoring states from the checkpoint path at ../../../user_data/checkpoints/kaggle_dog_active/active_checkpoints_BadgeSampling/BadgeSampling_round_10-epoch=59-train_loss=0.20083.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ../../../user_data/checkpoints/kaggle_dog_active/active_checkpoints_BadgeSampling/BadgeSampling_round_10-epoch=59-train_loss=0.20083.ckpt
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

Restoring states from the checkpoint path at ../../../user_data/checkpoints/kaggle_dog_active/active_checkpoints_BadgeSampling/BadgeSampling_round_10-epoch=69-train_loss=0.19852.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ../../../user_data/checkpoints/kaggle_dog_active/active_checkpoints_BadgeSampling/BadgeSampling_round_10-epoch=69-train_loss=0.19852.ckpt
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Predicting: 0it [00:00, ?it/s]

Restoring states from the checkpoint path at ../../../user_data/checkpoints/kaggle_dog_active/active_checkpoints_VarRatio/VarRatio_round_9-epoch=89-train_loss=0.25712.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ../../../user_data/checkpoints/kaggle_dog_active/active_checkpoints_VarRatio/VarRatio_round_9-epoch=89-train_loss=0.25712.ckpt
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

Restoring states from the checkpoint path at ../../../user_data/checkpoints/kaggle_dog_active/active_checkpoints_VarRatio/VarRatio_round_9-epoch=99-train_loss=0.29436.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at ../../../user_data/checkpoints/kaggle_dog_active/active_checkpoints_VarRatio/VarRatio_round_9-epoch=99-train_loss=0.29436.ckpt
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

In [15]:
fds

NameError: name 'fds' is not defined

In [16]:
pd.read_csv('result.csv')

,Unnamed: 0,Name,Score
0,0,BadgeSampling_round_10-epoch=49-train_loss=0.1...,0.897890
1,1,BadgeSampling_round_10-epoch=59-train_loss=0.2...,0.907474
2,2,BadgeSampling_round_10-epoch=69-train_loss=0.1...,0.924411
3,3,BadgeSampling_round_10-epoch=79-train_loss=0.1...,0.915889
4,4,BadgeSampling_round_10-epoch=89-train_loss=0.1...,0.920394
...,...,...,...
325,325,VarRatio_round_9-epoch=59-train_loss=0.28004.ckpt,0.948354
326,326,VarRatio_round_9-epoch=69-train_loss=0.27591.ckpt,0.930396
327,327,VarRatio_round_9-epoch=79-train_loss=0.26708.ckpt,0.931960
328,328,VarRatio_round_9-epoch=89-train_loss=0.25712.ckpt,0.935661


In [ ]:
# import re
# i = 0
# for stg in strategies:
#     filenames = os.listdir(ckpt_folder_root+'kaggle_dog_active/'+ stg)
#     pattern = r'round_(\d+)-epoch=(\d+)-train_loss'
# 
#     filtered_filenames = [
#         filename for filename in filenames
#         if int(re.search(pattern, filename).group(1)) >= 8 and int(re.search(pattern, filename).group(2)) >= 40
#     ]
#     
#     # Print the filtered filenames
#     for file in filtered_filenames:
#         print(file,i)
#         i+=1
#     # for ckpt in saved_ckpts:
#     #     print(ckpt)

In [ ]:
df = pd.DataFrame(columns = ['Name', 'Score'])
df.loc[len(df.index)] = ['test', 0.1]
df

In [ ]:
labels = pd.read_csv('SzDetectionAnswerKey.csv')

In [ ]:
fds

In [ ]:
output = torch.vstack(output_list)
S = output[:,0] + output[:,1]
E = output[:,0]

fpr, tpr, thresholds = roc_curve(labels.iloc[:13641][' seizure'], S)
S_roc_auc = auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(labels.iloc[:13641][' early '], E)
E_roc_auc = auc(fpr, tpr)
(S_roc_auc+E_roc_auc)/2

In [ ]:
len(S)

In [ ]:
E_roc_auc

In [ ]:
S_roc_auc

In [ ]:
(S_roc_auc+E_roc_auc)/2

In [ ]:
np.argmax(output,axis=1)

In [ ]:
len(torch.sort(m(output)[:, 0][inds])[0])

In [ ]:
sklearn.metrics.accuracy_score(np.argmax(m(output), axis=1), target)

In [ ]:
clf_report = sklearn.metrics.classification_report(np.argmax(m(output), axis=1), target, digits=6)

print(f"Classification Report : \n{clf_report}")

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [ ]:
pca_comp_n = 30
batch_size = 32

pca = PCA(n_components=pca_comp_n, copy=True).fit(emb)
p = pca.transform(emb)

# ind = np.random.choice(len(emb), 10000)
#
tsne = TSNE(n_components=2, verbose=1, perplexity=75, random_state=142, init='pca')
z = tsne.fit_transform(emb)



In [ ]:
ictal_pre_inds = np.where(target == 0)[0]
ictal_inds = np.where(target == 1)[0]
interictal_inds = np.where(target == 2)[0]

In [ ]:
spc = z

plt.figure(figsize=(10, 8))
plt.scatter(spc[interictal_inds,0],spc[interictal_inds,1],c='gold',label= 'interictal')
plt.scatter(spc[ictal_inds, 0], spc[ictal_inds, 1], c='royalblue', label='ictal')
plt.scatter(spc[ictal_pre_inds, 0], spc[ictal_pre_inds, 1], c='black', label='ictal')
plt.title('Swav Embedding t-SNE')
plt.xlabel('comp 1')
plt.ylabel("comp 2" )
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import RocCurveDisplay
target, np.argmax(output,axis=1)

RocCurveDisplay.from_predictions(
    target,
    np.argmax(output,axis=1),
    color="darkorange",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("One-vs-Rest ROC curves:\nVirginica vs (Setosa & Versicolor)")
plt.legend()
plt.show()

In [ ]:
output[:, 1]

In [ ]:

output = torch.argmax(output, dim=1)
output = output.detach().cpu().numpy()
target = target.squeeze().detach().cpu().numpy()

In [ ]:
import sklearn

clf_report = sklearn.metrics.classification_report(output, target, digits=6)

print(f"Classification Report : \n{clf_report}")

In [ ]:
for batch, label in tqdm(val_dataloader):
    batch = batch.to(device)
    label = label.to(device)
    label = F.one_hot(label).squeeze()
    outputs = model(batch)
    print(batch)
    loss = sigmoid_focal_loss(pred.float(), label.float(), alpha=0.5, gamma=8, reduction='mean')
    print(loss)
    break

In [ ]:
# import copy
# import torch
# import torchvision
# from torch import nn
#
# from lightly.data import DINOCollateFunction, LightlyDataset
# from lightly.loss import DINOLoss
# from lightly.models.modules import DINOProjectionHead
# from lightly.models.utils import deactivate_requires_grad, update_momentum
# from lightly.utils.scheduler import cosine_schedule
#
#
# class DINO(torch.nn.Module):
#     def __init__(self, backbone, input_dim):
#         super().__init__()
#         self.student_backbone = backbone
#         self.student_head = DINOProjectionHead(
#             input_dim, 512, 64, 2048, freeze_last_layer=1
#         )
#         self.teacher_backbone = copy.deepcopy(backbone)
#         self.teacher_head = DINOProjectionHead(input_dim, 512, 64, 2048)
#         deactivate_requires_grad(self.teacher_backbone)
#         deactivate_requires_grad(self.teacher_head)
#
#     def forward(self, x):
#         y = self.student_backbone(x).flatten(start_dim=1)
#         z = self.student_head(y)
#         return z
#
#     def forward_teacher(self, x):
#         y = self.teacher_backbone(x).flatten(start_dim=1)
#         z = self.teacher_head(y)
#         return z
#
#
# resnet = torchvision.models.resnet18()
# backbone = nn.Sequential(*list(resnet.children())[:-1])
# input_dim = 512
# # instead of a resnet you can also use a vision transformer backbone as in the
# # original paper (you might have to reduce the batch size in this case):
# # backbone = torch.hub.load('facebookresearch/dino:main', 'dino_vits16', pretrained=False)
# # input_dim = backbone.embed_dim
#
# model = DINO(backbone, input_dim)
#
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)
#
# # # we ignore object detection annotations by setting target_transform to return 0
# # pascal_voc = torchvision.datasets.VOCDetection(
# #     "datasets/pascal_voc", download=True, target_transform=lambda t: 0
# # )
# # dataset = LightlyDataset.from_torch_dataset(pascal_voc)
# # # or create a dataset from a folder containing images or videos:
# # # dataset = LightlyDataset("path/to/folder")
#
# collate_fn = DINOCollateFunction(solarization_prob = 0, hf_prob = 0,vf_prob = 0,rr_prob=0,cj_prob=0,random_gray_scale=0)
#
# dataloader = torch.utils.data.DataLoader(
#     train_set,
#     batch_size=64,
#     collate_fn=collate_fn,
#     shuffle=True,
#     drop_last=True,
#     num_workers=1,
# )
#
# criterion = DINOLoss(
#     output_dim=2048,
#     warmup_teacher_temp_epochs=5,
# )
# # move loss to correct device because it also contains parameters
# criterion = criterion.to(device)
#
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#
# epochs = 10
#
# print("Starting Training")
# for epoch in range(epochs):
#     total_loss = 0
#     momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
#     for views, _, _ in tqdm(dataloader):
#         update_momentum(model.student_backbone, model.teacher_backbone, m=momentum_val)
#         update_momentum(model.student_head, model.teacher_head, m=momentum_val)
#         views = [view.to(device) for view in views]
#         global_views = views[:2]
#         teacher_out = [model.forward_teacher(view) for view in global_views]
#         student_out = [model.forward(view) for view in views]
#         loss = criterion(teacher_out, student_out, epoch=epoch)
#         total_loss += loss.detach()
#         loss.backward()
#         # We only cancel gradients of student head.
#         model.student_head.cancel_last_layer_gradients(current_epoch=epoch)
#         optimizer.step()
#         optimizer.zero_grad()
#
#     avg_loss = total_loss / len(dataloader)
#     print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

In [ ]:
augmentation = T.Compose([
    T.ToPILImage(),
    T.Resize((256, 512), interpolation=T.InterpolationMode.NEAREST),
    T.RandomApply([T.ColorJitter()], p=0.5),
    T.RandomApply([T.GaussianBlur(kernel_size=(3, 3))], p=0.5),
    T.RandomInvert(p=0.2),
    T.RandomPosterize(4, p=0.2),
])

data = ictal_data_X[0]

channel_index = np.arange(data.shape[0])
np.random.shuffle(channel_index)
data = data[channel_index]
data = torch.from_numpy(data).clone()
data = data.repeat(3, 1, 1)
data = augmentation(data)
data

In [ ]:
channel_index

In [ ]:
data[channel_index]

In [ ]:
data

In [ ]:
#
# print("Starting Training")
# for epoch in range(50):
#     total_loss = 0
#     i = 0
#     for batch, label in tqdm(dataloader):
#         batch = batch.to(device)
#         # print(type(batch))
#         label = label.to(device)
#         label = F.one_hot(label).squeeze()
#         outputs = model(batch)
#         loss = sigmoid_focal_loss(outputs.float(),label.float(), alpha = 0.25, gamma = 7,reduction = 'mean')
#         total_loss += loss.detach()
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#
#     avg_loss = total_loss / len(dataloader)
#     torch.save({
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': avg_loss,
#             }, 'ckpt/checkpoint'+str(epoch)+'.pth')
#
#     print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")    